<a href="https://colab.research.google.com/github/u-masao/vector-search-tutorial-for-japanese/blob/main/Elasticsearch_Installer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elasticsearch Installer

In [3]:
ES_VERSION='8.18.2'

8.18.2


In [19]:
# スクリプト用に環境変数を設定
import os

os.environ['ES_VERSION']=ES_VERSION
os.environ['ES_FILENAME']=f"elasticsearch-{ES_VERSION}-linux-x86_64.tar.gz"
os.environ['ES_DIRNAME']=f"elasticsearch-{ES_VERSION}"

In [20]:
# Java ランタイムのインストール（yum、apt-get 自動切換え）

%%bash

# スクリプトの途中でエラーが発生したら、処理を中断する
set -e

# --- 共通の変数定義 ---

echo "--- 1. 環境を判定し、Java (JDK) をインストールします ---"

# 'command -v' を使って yum と apt-get の存在を確認し、処理を分岐
if command -v yum &> /dev/null; then
    # --- SageMaker (Amazon Linux) 向けの処理 ---
    echo "Yum-based system (SageMaker/Amazon Linux) を検出しました。"
    sudo yum install -y java-11-openjdk-headless
elif command -v apt-get &> /dev/null; then
    # --- Google Colab (Debian/Ubuntu) 向けの処理 ---
    echo "Apt-based system (Google Colab/Debian) を検出しました。"
    # 最初にパッケージリストを更新するのが良い作法
    sudo apt-get update -qq
    sudo apt-get install -y openjdk-11-jdk-headless -qq
else
    # どちらのパッケージマネージャーも見つからない場合
    echo "エラー: yum も apt-get も見つかりません。Javaをインストールできません。" >&2
    exit 1
fi
echo "✅ Javaのインストールが完了しました。"

--- 1. 環境を判定し、Java (JDK) をインストールします ---
Apt-based system (Google Colab/Debian) を検出しました。
✅ Javaのインストールが完了しました。


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [21]:
# ダウンロード、展開、設定、ユーザー作成

%%bash

set -e

echo -e "\n--- 2. Elasticsearch ${ES_VERSION} のダウンロードと展開 ---"

if [ ! -f "${ES_FILENAME}" ]; then
    wget "https://artifacts.elastic.co/downloads/elasticsearch/${ES_FILENAME}" -q
    echo "✅ ダウンロードが完了しました。"
else
    echo "✅ ファイルは既に存在します。ダウンロードをスキップします。"
fi
ls
tar -xzf "${ES_FILENAME}"
echo "✅ Elasticsearchの展開が完了しました。"

echo -e "\n--- 3. 設定ファイルの追記 ---"

CONFIG_FILE="./${ES_DIRNAME}/config/elasticsearch.yml"

# 設定が重複しないように、一度ファイルから該当行を削除してから追記する
grep -q "xpack.security.enabled" "${CONFIG_FILE}" && sed -i "/xpack.security.enabled/d" "${CONFIG_FILE}" || true
echo "xpack.security.enabled: false" >> "${CONFIG_FILE}"
echo "✅ セキュリティ機能を無効にする設定を追記しました。"

grep -q "xpack.ml.enabled" "${CONFIG_FILE}" && sed -i "/xpack.ml.enabled/d" "${CONFIG_FILE}" || true
echo "xpack.ml.enabled: false" >> "${CONFIG_FILE}"
echo "✅ Machine Learning機能を無効にする設定を追記しました。"

echo -e "\n--- 4. 実行ユーザーの作成と権限設定 ---"
# sudoコマンドはColab(root環境)でも問題なく実行できるため、共通化できる
# 既にユーザーが存在する場合のエラーを回避するため '|| true' を追加
sudo useradd -m -s /bin/bash elastic || true
sudo chown -R elastic:elastic "./${ES_DIRNAME}/"
echo "✅ 実行ユーザーの作成と権限設定が完了しました。"


--- 2. Elasticsearch 8.18.2 のダウンロードと展開 ---
✅ ダウンロードが完了しました。
elasticsearch-8.13.4
elasticsearch-8.18.2-gomi
elasticsearch-8.18.2-linux-x86_64.tar.gz
sample_data
✅ Elasticsearchの展開が完了しました。

--- 3. 設定ファイルの追記 ---
✅ セキュリティ機能を無効にする設定を追記しました。
✅ Machine Learning機能を無効にする設定を追記しました。

--- 4. 実行ユーザーの作成と権限設定 ---
✅ 実行ユーザーの作成と権限設定が完了しました。


+ echo -e '\n--- 2. Elasticsearch 8.18.2 のダウンロードと展開 ---'
+ '[' '!' -f elasticsearch-8.18.2-linux-x86_64.tar.gz ']'
+ wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.18.2-linux-x86_64.tar.gz -q
+ echo '✅ ダウンロードが完了しました。'
+ ls
+ tar -xzf elasticsearch-8.18.2-linux-x86_64.tar.gz
+ echo '✅ Elasticsearchの展開が完了しました。'
+ echo -e '\n--- 3. 設定ファイルの追記 ---'
+ CONFIG_FILE=./elasticsearch-8.18.2/config/elasticsearch.yml
+ grep -q xpack.security.enabled ./elasticsearch-8.18.2/config/elasticsearch.yml
+ true
+ echo 'xpack.security.enabled: false'
+ echo '✅ セキュリティ機能を無効にする設定を追記しました。'
+ grep -q xpack.ml.enabled ./elasticsearch-8.18.2/config/elasticsearch.yml
+ true
+ echo 'xpack.ml.enabled: false'
+ echo '✅ Machine Learning機能を無効にする設定を追記しました。'
+ echo -e '\n--- 4. 実行ユーザーの作成と権限設定 ---'
+ sudo useradd -m -s /bin/bash elastic
useradd: user 'elastic' already exists
+ true
+ sudo chown -R elastic:elastic ./elasticsearch-8.18.2/
+ echo '✅ 実行ユーザーの作成と権限設定が完了しました。'


In [22]:
%%bash
# スクリプトの途中でエラーが発生したら、処理を中断する
set -e

echo -e "\n--- 5. Elasticsearchの起動 ---"
# sudo su ... も同様に共通化
sudo su elastic -c "./${ES_DIRNAME}/bin/elasticsearch -d"
echo "🚀 Elasticsearchをバックグラウンドで起動します... (完了まで30秒ほどかかります)"

# 6. 起動完了まで待機
sleep 30

echo -e "\n--- 7. 起動確認 ---"
curl -X GET "localhost:9200/"


--- 5. Elasticsearchの起動 ---
[2025-06-19T14:24:24,372][INFO ][o.e.n.j.JdkVectorLibrary ] [eaae98a487ce] vec_caps=1
[2025-06-19T14:24:24,419][INFO ][o.e.n.NativeAccess       ] [eaae98a487ce] Using native vector library; to disable start with -Dorg.elasticsearch.nativeaccess.enableVectorLibrary=false
[2025-06-19T14:24:24,460][INFO ][o.e.n.NativeAccess       ] [eaae98a487ce] Using [jdk] native provider and native methods for [Linux]
[2025-06-19T14:24:24,588][INFO ][o.a.l.i.v.PanamaVectorizationProvider] [eaae98a487ce] Java vector incubator API enabled; uses preferredBitSize=256; FMA enabled
[2025-06-19T14:24:24,645][INFO ][o.e.b.Elasticsearch      ] [eaae98a487ce] Bootstrapping Entitlements
[2025-06-19T14:24:28,947][INFO ][o.e.n.Node               ] [eaae98a487ce] version[8.18.2], pid[17826], build[tar/c6b8d8d951c631db715485edc1a74190cdce4189/2025-05-23T10:07:06.210694702Z], OS[Linux/6.1.123+/amd64], JVM[Oracle Corporation/OpenJDK 64-Bit Server VM/24/24+36-3646]
[2025-06-19T14:24:28,948][

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   537  100   537    0     0   5711      0 --:--:-- --:--:-- --:--:--  5774


In [25]:
%%bash
ps axuww | grep -i elastic

root       18528  0.0  0.0   6484  2160 ?        S    14:26   0:00 grep -i elastic


In [26]:
%%bash
sudo pkill -f elasticsearch

In [ ]:
%%bash
ps axuww | grep -i elastic